# Primer Job 
Escribe en el archivo format para cada jugador su puntaje promedio y su puntaje historico nuevo y anterior (en la primera vuelta son iguales) y si alguien lo reto

In [75]:
from MRE import Job
partidasDir = "./partidas/"
promedioDir = "../punto2/contabilizado/"
outputDir = "./format/"

In [76]:
def fmapBase(key, value, context):
    lista = value.split()
    ph = 1
    context.write((int(key), "PP"), ((float(lista[0]) + 1)/(float(lista[1])+1),ph))

def fmapPartida(retador, value, context):
    lista = value.split("\t")
    retado = lista[0]
    context.write((int(retado), "SENT TO"), int(retador))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1
    
def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP y PH es el primero 
        return -1
    else:
        return 1 #Caso Contribucion
    
def fred(key, values, context):
    estructura = next(values) 
    # Estructura esperada: [PP_i , PH_viejo]
    pp_i = float(estructura[0])
    ph = estructura[1]
    
    
    for v in values: 
        retador = v
        context.write(key[0], (pp_i, puntaje_heroico_inicial, ph, retador))

    
puntaje_heroico_inicial = 1

jobCount = Job(promedioDir, outputDir, fmapBase, fred)
jobCount.setParams(puntaje_heroico_inicial)
jobCount.addInputPath(partidasDir, fmapPartida)
jobCount.setShuffleCmp(cmpShort)
jobCount.setSortCmp(cmpShuffle)

success = jobCount.waitForCompletion()

print(success)

True


# Segundo job  = iterador
Toma la salida del job anterior para saber los datos de cada jugador contribuir al calculo de quienes lo retaron (si nadie lo reto no envia info)

In [ ]:

def fmapJugador(key, value, context):
    lista = value.split("\t")
    promedio = lista[0]
    ph_inicial = lista[1]
    ph_viejo = lista[2]
    context.write((int(key), "DATOS"), (promedio, ph_inicial))
    if len(lista) > 3:  # Evitar casos sin retadores (este caso se da si un jugador no fue retado por a nadie)
        retador = lista[3]
        context.write((int(retador), "CONTRIBUCION"), (promedio, ph_inicial))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1
    
def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "DATOS"): #PP y PH es el primero 
        return -1
    else:
        return 1 #Caso Contribucion

def fred(key, values, context):
    estructura = next(values) 
    # Estructura esperada: [PP_i , PH_viejo]
    
    pp_i = float(estructura[0])
    ph_viejo = float(estructura[1])
    
    
    acumulador = 0
    for v in values:
        # v viene del Mapper como (PP_j, PH_j)
        pp_j = float(v[0])
        ph_j = float(v[1])
        acumulador += ph_j * pp_i / pp_j
    
    nuevo_ph = (alfa * acumulador) + (1 - alfa)
    
    # Emitimos manteniendo la estructura para la siguiente iteración
    # Formato: ID \t PP \t PH_Nuevo \t PH_Viejo
    context.write(key[0], (pp_i, nuevo_ph, ph_viejo))

# JOB INTERMEDIO QUE SOLO CALCULA EL MAE ENTRE DOS ITERACIONES
def map_mae(key, value, context):
    # La estructura de value es: [PP, PH_Nuevo, PH_Viejo]
    datos = value.split("\t")
        
    ph_nuevo = float(datos[1])
    ph_viejo = float(datos[2])
    
    # Calculamos el error absoluto: |PH_nuevo - PH_viejo|
    error_absoluto = abs(ph_nuevo - ph_viejo)
    
    # Enviamos a una clave fija para que un Reducer calcule el promedio final
    # Emitimos (error, 1) para poder calcular el promedio (Suma / Total)
    context.write("GLOBAL_MAE", (error_absoluto, 1))

def red_mae(key, values, context):
    suma_error_absoluto = 0
    conteo_jugadores = 0
    
    for err, count in values:
        suma_error_absoluto += err
        conteo_jugadores += count
    print(suma_error_absoluto)
    print(conteo_jugadores)
    # MAE = Suma de errores absolutos / N
    mae_final = suma_error_absoluto / conteo_jugadores if conteo_jugadores > 0 else 0
    
    context.write("RESULTADO_MAE", mae_final)


# JOB PRINCIPAL DE ITERACIÓN
def fmapBase(key, value, context):
    lista = value.split()
    context.write((int(key), "PP"), (float(lista[0]), float(lista[1]), float(lista[2])))

def fmapPartida(retador, value, context):
    lista = value.split("\t")
    retado = lista[0]
    context.write((int(retado), "SENT TO"), int(retador))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1
    
def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP y PH es el primero 
        return -1
    else:
        return 1 #Caso Contribucion
    
def fredBase(key, values, context):
    estructura = next(values) 
    # Estructura esperada: [PP_i , PH_viejo]
    pp_i = float(estructura[0])
    ph = estructura[1]
    ph_viejo = estructura[2]
    
    
    for v in values: 
        retador = v
        context.write(key[0], (pp_i, ph, ph_viejo, retador))

# Parámetros iniciales
alfa = 0.1
error_tolerado = 0.001
max_iter = 20
input_dir = "./format/" # Salida del Job anterior (ID \t [PP, PH, {Retados}])

for i in range(max_iter):
    output_dir = f"./iteracion_{i}/"
    
    # Configuramos el Job
    job = Job(input_dir, output_dir, fmapJugador, fred)
    job.setParams(alfa)
    job.setShuffleCmp(cmpShort)
    job.setSortCmp(cmpShuffle)

    success = job.waitForCompletion()
    
    # --- CÁLCULO DE CONVERGENCIA (MSE) ---
    mae_dir = f"./mae_iter_{i}/"
    
    # 1. Ejecutar Job de cálculo de PH (el que ya tienes)
    # 2. Ejecutar Job de MAE
    jobMAE = Job(output_dir, mae_dir, map_mae, red_mae)
    jobMAE.waitForCompletion()
    
    # 3. Leer el resultado del MAE desde el Driver
    # (El driver solo lee un archivo de texto de pocos bytes)
    try:
        with open(f"{mae_dir}/output.txt", "r") as f:
            # Formato esperado: RESULTADO_MAE \t 0.00045
            linea = f.read().split("\t")
            mae_actual = float(linea[1])
            
        print(f">>> Iteración {i} - MAE: {mae_actual:.6f}")
        
        if mae_actual < error_tolerado:
            print(f"Convergió en iteración {i} con MAE {mae_actual}")
            break
            
    except Exception as e:
        print(f"Error leyendo MAE: {e}")
        break
    
    # JOB DE JOIN ENTRE PARTIDAS Y JUGADORES PARA LA SIGUIENTE ITERACIÓN
    intermedio_dir = f"./intermedio_iter_{i}/"
    jobJoin = Job(output_dir, intermedio_dir, fmapBase, fredBase)
    jobJoin.addInputPath(partidasDir, fmapPartida)
    jobJoin.setShuffleCmp(cmpShort)
    jobJoin.setSortCmp(cmpShuffle)

    success = jobJoin.waitForCompletion()
    
    # El output de esta vuelta es el input de la siguiente
    input_dir = intermedio_dir

print(success)



4889.171911287918
112
>>> Iteración 0 - MAE: 43.653321
16697.961871309395
112
>>> Iteración 1 - MAE: 149.088945
34745.7304426075
112
>>> Iteración 2 - MAE: 310.229736
42213.38883002522
112
>>> Iteración 3 - MAE: 376.905257
69222.16460791159
112
>>> Iteración 4 - MAE: 618.055041
107412.41066179574
112
>>> Iteración 5 - MAE: 959.039381
189432.63157370707
112
>>> Iteración 6 - MAE: 1691.362782
322954.0532766071
112
>>> Iteración 7 - MAE: 2883.518333
536862.0741115605
112
>>> Iteración 8 - MAE: 4793.411376
877060.7873425758
112
>>> Iteración 9 - MAE: 7830.899887
1570428.231842245
112
>>> Iteración 10 - MAE: 14021.680641
2127644.6262834705
112
>>> Iteración 11 - MAE: 18996.827020
3896229.163199211
112
>>> Iteración 12 - MAE: 34787.760386
6130703.692305274
112
>>> Iteración 13 - MAE: 54738.425824
9470372.819059338
112
>>> Iteración 14 - MAE: 84556.900170
14408227.865552774
112
>>> Iteración 15 - MAE: 128644.891657
21949397.562279392
112
>>> Iteración 16 - MAE: 195976.763949
26828223.81441313

In [78]:
# JOB INTERMEDIO QUE SOLO CALCULA EL MAE ENTRE DOS ITERACIONES
def map_mae(key, value, context):
    # Asumiendo que la estructura de value es: [PP, {Retados}, PH_Nuevo, PH_Viejo]
    # Si 'value' viene de disco como string, hay que parsearlo
    datos = value.split("\t")
        
    ph_nuevo = float(datos[2])
    ph_viejo = float(datos[3])
    
    # Calculamos el error absoluto: |PH_nuevo - PH_viejo|
    error_absoluto = abs(ph_nuevo - ph_viejo)
    
    # Enviamos a una clave fija para que un Reducer calcule el promedio final
    # Emitimos (error, 1) para poder calcular el promedio (Suma / Total)
    context.write("GLOBAL_MAE", (error_absoluto, 1))

def red_mae(key, values, context):
    suma_error_absoluto = 0
    conteo_jugadores = 0
    
    for err, count in values:
        suma_error_absoluto += err
        conteo_jugadores += count
    
    # MAE = Suma de errores absolutos / N
    mae_final = suma_error_absoluto / conteo_jugadores if conteo_jugadores > 0 else 0
    
    context.write("RESULTADO_MAE", mae_final)


# Parámetros iniciales
alfa = 0.1
error_tolerado = 0.001
max_iter = 20
input_dir = "./format/" # Salida del Job anterior (ID \t [PP, PH])
partidas_dir = "./partidas/"

for i in range(max_iter):
    output_dir = f"./iteracion_{i}/"
    
    # Configuramos el Job
    job = Job(input_dir, output_dir, fmap, fred)
    job.setParams(alfa)
    job.setShuffleCmp(cmpShort)
    job.setSortCmp(cmpShuffle)
    
    success = job.waitForCompletion()
    
    # --- CÁLCULO DE CONVERGENCIA (MSE) ---
    mae_dir = f"./mae_iter_{i}/"
    
    # 1. Ejecutar Job de cálculo de PH (el que ya tienes)
    # 2. Ejecutar Job de MAE
    jobMAE = Job(output_dir, mae_dir, map_mae, red_mae)
    jobMAE.waitForCompletion()
    
    # 3. Leer el resultado del MAE desde el Driver
    # (El driver solo lee un archivo de texto de pocos bytes)
    try:
        with open(f"{mae_dir}/output.txt", "r") as f:
            # Formato esperado: RESULTADO_MAE \t 0.00045
            linea = f.read().split("\t")
            mae_actual = float(linea[1])
            
        print(f">>> Iteración {i} - MAE: {mae_actual:.6f}")
        
        if mae_actual < error_tolerado:
            print(f"Convergió en iteración {i} con MAE {mae_actual}")
            break
            
    except Exception as e:
        print(f"Error leyendo MAE: {e}")
        break
    
    # El output de esta vuelta es el input de la siguiente
    input_dir = output_dir

print(success)

NameError: name 'fmap' is not defined

Tercer Job saca top 10 

In [ ]:
inputDir = "./format/"
outputDir = "./out/"

In [ ]:
import heapq

def fmap(key, value, context):
    lista = value.split("\t")
    context.write(1, (key, float(lista[-1])))

def fcombiner(key, values, context):
    # simple combiner: sum counts locally
    top_maximo = []
    
    for v in values:
        elemento = (v[1],v[0])
        if len(top_maximo) < 10:
            # Si hay menos de 10 elementos, simplemente lo añadimos al heap.
            heapq.heappush(top_maximo, elemento)
        else:
            # Si el nuevo score (elemento[0]) es MAYOR que el elemento MÁS PEQUEÑO 
            # del heap (top_maximo[0]), reemplazamos el pequeño.
            if elemento[0] > top_maximo[0][0]:
                heapq.heapreplace(top_maximo, elemento)
    top_10_final = sorted(top_maximo, key=lambda e: e[0], reverse=True)
    for (maximo, clave) in top_10_final:
        context.write(key, (clave,maximo))

def fred(key, values, context):
    # simple combiner: sum counts locally
    top_maximo = []
    
    for v in values:
        elemento = (v[1],v[0])
        if len(top_maximo) < 10:
            # Si hay menos de 10 elementos, simplemente lo añadimos al heap.
            heapq.heappush(top_maximo, elemento)
        else:
            # Si el nuevo score (elemento[0]) es MAYOR que el elemento MÁS PEQUEÑO 
            # del heap (top_maximo[0]), reemplazamos el pequeño.
            if elemento[0] > top_maximo[0][0]:
                heapq.heapreplace(top_maximo, elemento)
    top_10_final = sorted(top_maximo, key=lambda e: e[0], reverse=True)
    for (maximo, clave) in top_10_final:
        context.write(clave, maximo)

jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCountBoth.setCombiner(fcombiner)
success = jobCountBoth.waitForCompletion()

print(success)

True
